## Домашнее задание к 8 семинару

На сайте запущен А/В тест с целью увеличить доход. В приложенном excel файле вы найдете сырые данные по результатам эксперимента – user_id, тип выборки variant_name и доход принесенный пользователем revenue.
Проанализируйте результаты эксперимента и напишите свои рекомендации менеджеру.

### Загрузка библиотек

In [274]:
from typing import Union
from tqdm import tqdm

import pandas as pd
import numpy as np

from scipy import stats
from statsmodels.stats.meta_analysis import effectsize_smd
from statsmodels.stats import proportion
from statsmodels.stats.power import tt_ind_solve_power
from statsmodels.stats.power import zt_ind_solve_power


### Подготовка данных

In [275]:
data = pd.read_excel('data/gb_sem_8_hm.xlsx');

data.head(10)

c:\Users\1\AppData\Local\Programs\Python\Python38\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning:

Unknown extension is not supported and will be removed



,USER_ID,VARIANT_NAME,REVENUE
0,737,variant,0.0
1,2423,control,0.0
2,9411,control,0.0
3,7311,control,0.0
4,6174,variant,0.0
5,2380,variant,0.0
6,2849,control,0.0
7,9168,control,0.0
8,6205,variant,0.0
9,7548,control,0.0


In [276]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   USER_ID       10000 non-null  int64  
 1   VARIANT_NAME  10000 non-null  object 
 2   REVENUE       10000 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 234.5+ KB


#### А/В варианты отдельно

In [277]:
Control = data[data['VARIANT_NAME'] == 'control'].copy(deep=True)
Variant = data[data['VARIANT_NAME'] == 'variant'].copy(deep=True)

#### Статистические данные по базовому варианту

In [496]:

Control.describe()

,USER_ID,REVENUE
count,4984.000000,4984.000000
mean,4989.436798,0.129013
std,2905.145109,3.007524
min,2.000000,0.000000
25%,2466.000000,0.000000
50%,4964.500000,0.000000
75%,7576.250000,0.000000
max,10000.000000,196.010000


#### Статистические данные по тестовому варианту

In [510]:
Variant.describe()

,USER_ID,REVENUE
count,5016.000000,5016.000000
mean,4972.776914,0.070070
std,2876.320625,1.314802
min,3.000000,0.000000
25%,2476.500000,0.000000
50%,4958.500000,0.000000
75%,7415.250000,0.000000
max,10000.000000,58.630000


### Применение статистических критериев

In [511]:
def continious_result(Control: pd.DataFrame,
                      Variant: pd.DataFrame,
                      column: str,
                      n_iters: int = 10_000) -> pd.DataFrame:
    # Статистика по выборкам
    size = Control.loc[:, column].shape[0]

    Control_mean = Control.loc[:, column].mean()
    Variant_mean = Variant.loc[:, column].mean()
    
    Control_std = Control.loc[:, column].std(ddof=1)
    Variant_std = Variant.loc[:, column].std(ddof=1)
    
    
    # Бутсрап
    booted_diff = []
    for _ in tqdm(range(n_iters)):
        Control_sample = Control.loc[:, column].sample(n=size, replace=True).values
        Variant_sample = Variant.loc[:, column].sample(n=size, replace=True).values
        booted_diff.append(np.mean(Control_sample - Variant_sample))
    
    # Считаем статистику после бустрапа
    md_ci, std_ci = np.mean(booted_diff), np.std(booted_diff, ddof=1)
    left_ci, right_ci = np.percentile(booted_diff, [2.5, 97.5])
    p_value_ci = 2 * (1 - stats.norm.cdf(np.abs(md_ci / std_ci)))
    
    # Считаем мощность эксперимента
    effect_size, _ = effectsize_smd(mean1=Variant_mean, sd1=Variant_std, nobs1=size,
                                    mean2=Control_mean, sd2=Control_std, nobs2=size)
    power = tt_ind_solve_power(effect_size=effect_size,
                               nobs1=size,
                               alpha=.05,
                               power=None,
                               ratio=1)
    # Формируем отчёт 
    result = pd.DataFrame({'effect_size': effect_size,
                           'alpha': p_value_ci, 
                           'beta': (1-power),
                           'CI': f'[{np.round(left_ci, 3)}, {np.round(right_ci, 3)}]',
                           'difference': md_ci,},
                          index=[column]) 
    return result



In [512]:
def proportion_result(Control: pd.DataFrame,
                      Variant: pd.DataFrame,
                      column: str,
                      n_iters: int = 10_000) -> pd.DataFrame:
    # Вероятность событий
    size = Control.loc[:, column].shape[0]
    prop_Control = Control.loc[:, column].sum() / size
    prop_Variant = Variant.loc[:, column].sum() / size
    
    # Бутсрап
    booted_diff = []
    for _ in tqdm(range(n_iters)):
        Control_sample = stats.bernoulli.rvs(p=prop_Control, size=size)
        Variant_sample = stats.bernoulli.rvs(p=prop_Variant, size=size)
        booted_diff.append(np.mean(Control_sample - Variant_sample))
    
    # Считаем статистику после бустрапа
    md_ci, std_ci = np.mean(booted_diff), np.std(booted_diff, ddof=1)
    left_ci, right_ci = np.percentile(booted_diff, [2.5, 97.5])
    p_value_ci = 2 * (1 - stats.norm.cdf(np.abs(md_ci / std_ci)))
    
    # Считаем мощность эксперимента
    effect_size = proportion.proportion_effectsize(prop_Control, prop_Variant)
    
    power = zt_ind_solve_power(effect_size=effect_size,
                               nobs1=size,
                               alpha=.05,
                               power=None,
                               ratio=1)
    # Формируем отчёт 
    result = pd.DataFrame({'effect_size': effect_size,
                           'alpha': p_value_ci, 
                           'beta': (1-power),
                           'CI': f'[{np.round(left_ci, 3)}, {np.round(right_ci, 3)}]',
                           'difference': md_ci,},
                          index=[column]) 
    return result

#### Проверяем размеры выборок

In [513]:
data1 = data['VARIANT_NAME'].value_counts();
data1

variant    5016
control    4984
Name: VARIANT_NAME, dtype: int64

Выборки примерно одинаковые.

#### Определяем количество покупок

In [514]:
data3 = Variant['REVENUE'].value_counts().reset_index()
data3.head()

,index,REVENUE
0,0.00,4944
1,1.25,5
2,1.01,4
3,0.04,3
4,1.99,2


По тестовому варианту = 5016 (всего юзеров) - 4944 (юзеры не совершившие покупок) = 72.

Доля юзеров, совершивих покупки = 72/5016 = 0,0144.

In [515]:
data2 = Control['REVENUE'].value_counts().reset_index()
data2.head()

,index,REVENUE
0,0.00,4904
1,1.25,5
2,3.25,4
3,2.17,3
4,4.33,3


По базовому варианту = 4984 (всего юзеров) - 4904 (юзеры не совершившие покупок) = 80

Доля юзеров, совершивих покупки = 80/4984 = 0,0161.

#### Проверяем разницу в калькуляторе

Вопрос: Различается ли показатель успеха в двух группах?

Образец 1:	80/	4984 - 1,3 % – 2 %

Образец 2:	72/	5016 - 1,1 % – 1,8 %

Вердикт:нет существенной разницы (p = 0,49 ).

#### Определяем доход

In [516]:
data['REVENUE'].sum()

994.47

Общий доход = 994.47

In [517]:
Summa_variant = Variant['REVENUE'].sum()
Summa_variant

351.47

Доход по тестовому варианту = 351.47

Средний чек по тестовому варианту = 351,47/72 = 4,88.

In [518]:
Summa_control = Control['REVENUE'].sum()
Summa_control

643.0

Доход по контрольному варианту = 643.0


Средний чек по контрольному варианту = 643.0/80 = 8,04.

Средний чек по тестовому варианту уменьшился по сравнению с контрольным вариантом на 

4,88/8,04 *100% = 61%. 

In [519]:
import plotly.express as px

In [520]:
fig = px.histogram(data[data['REVENUE'] > 0],
                   x='REVENUE',
                   color = 'VARIANT_NAME',
                   title='avg_site_visits_distribution',
                   marginal = 'box',
                   nbins = 60,
                   barmode='overlay')
fig.show()

#### Метрика конверсия в покупку

In [521]:
def proportion_result(Control: pd.DataFrame,
                      Variant: pd.DataFrame,
                      column: str,
                      n_iters: int = 10_000) -> pd.DataFrame:
    # Вероятность событий
    size = Control.loc[:, column].shape[0]
    prop_Control = Control.loc[:, column].sum() / size
    prop_Variant = Variant.loc[:, column].sum() / size
    
    # Бутсрап
    booted_diff = []
    for _ in tqdm(range(n_iters)):
        Control_sample = stats.bernoulli.rvs(p=prop_Control, size=size)
        Variant_sample = stats.bernoulli.rvs(p=prop_Variant, size=size)
        booted_diff.append(np.mean(Control_sample - Variant_sample))
    
    # Считаем статистику после бустрапа
    md_ci, std_ci = np.mean(booted_diff), np.std(booted_diff, ddof=1)
    left_ci, right_ci = np.percentile(booted_diff, [2.5, 97.5])
    p_value_ci = 2 * (1 - stats.norm.cdf(np.abs(md_ci / std_ci)))
    
    # Считаем мощность эксперимента
    effect_size = proportion.proportion_effectsize(prop_Control, prop_Variant)
    
    power = zt_ind_solve_power(effect_size=effect_size,
                               nobs1=size,
                               alpha=.05,
                               power=None,
                               ratio=1)
    # Формируем отчёт 
    result = pd.DataFrame({'effect_size': effect_size,
                           'alpha': p_value_ci, 
                           'beta': (1-power),
                           'CI': f'[{np.round(left_ci, 3)}, {np.round(right_ci, 3)}]',
                           'difference': md_ci,},
                          index=[column]) 
    return result

In [522]:
proportion_result(Control, Variant, column='REVENUE')

,effect_size,alpha,beta,CI,difference
REVENUE,0.197226,0.0,1.554312e-15,"[0.047, 0.07]",0.058439


### Вывод: 
aнализ критериев показал, что статистически значимые отличия между контрольным вариантом и тестовым есть. Ключевые метрики значимо лучше в контрольном варианте.Тестовый вариант не оправдал ожиданий. Необходимо вернуться к прежней версии, чтобы не ухудшить финансовые показатели.